# BMI Calculator using LangGraph

This notebook demonstrates how to build a Body Mass Index (BMI) calculator using LangGraph's state management system. The program calculates BMI from weight and height inputs and categorizes the result according to standard BMI classifications.

## Overview

The BMI calculator is implemented as a graph workflow with two main steps:
1. **Calculate BMI**: Computes BMI using the formula: weight (kg) / height² (m²)
2. **Categorize BMI**: Labels the BMI result into standard categories (Underweight, Normal weight, Overweight, Obesity)

## Features
- Type-safe state management using TypedDict
- Sequential workflow processing with LangGraph
- Automatic BMI classification based on WHO standards

In [12]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict


## 1. Required Imports

We start by importing the necessary modules from LangGraph for building our state graph workflow.

In [19]:
class bmistate(TypedDict):
    """A type for the bmistate dictionary."""
    weight: float
    height: float
    bmi: float
    category: str


## 2. State Definition

We define the state structure using TypedDict to ensure type safety. The state contains:
- `weight`: Person's weight in kilograms
- `height`: Person's height in meters
- `bmi`: Calculated BMI value
- `category`: BMI classification category

In [7]:
def calculate_bmi(state:bmistate)-> bmistate:
    """Calculate BMI from weight and height."""
    state['bmi'] = state['weight'] / (state['height'] ** 2)
    return state

## 3. BMI Calculation Function

This function calculates the BMI using the standard formula:
**BMI = weight (kg) / height² (m²)**

The function takes the current state, performs the calculation, and updates the BMI field.

In [20]:
def label_bmi(state:bmistate)->bmistate:
    if state['bmi'] < 18.5:
        state['category'] = 'Underweight'
    elif 18.5 <= state['bmi'] < 24.9:
        state['category'] = 'Normal weight'     
    elif 25 <= state['bmi'] < 29.9:
        state['category'] = 'Overweight'    
    else:
        state['category'] = 'Obesity'
    return state

## 4. BMI Classification Function

This function categorizes the calculated BMI into standard WHO classifications:

| BMI Range | Category |
|-----------|----------|
| < 18.5 | Underweight |
| 18.5 - 24.9 | Normal weight |
| 25.0 - 29.9 | Overweight |
| ≥ 30.0 | Obesity |

In [21]:
#define graph
graph = StateGraph(bmistate)
#add node
graph.add_node("calculate_bmi", calculate_bmi)
graph.add_node("label_bmi", label_bmi)
#add edges
graph.add_edge(START,"calculate_bmi")
graph.add_edge("calculate_bmi","label_bmi")
graph.add_edge("label_bmi", END)
#compile
workflow = graph.compile()
#execute

## 5. Graph Workflow Definition

Here we create the LangGraph workflow that connects our functions:

1. **Graph Creation**: Initialize StateGraph with our BMI state type
2. **Node Addition**: Add the calculation and labeling functions as nodes
3. **Edge Definition**: Create the workflow sequence:
   - START → calculate_bmi → label_bmi → END
4. **Compilation**: Compile the graph into an executable workflow

In [25]:
init_state = bmistate(weight=105, height=1.70, bmi=0.0)
result=workflow.invoke(init_state)
print(result)

{'weight': 105, 'height': 1.7, 'bmi': 36.33217993079585, 'category': 'Obesity'}


## 7. Results and Conclusion

The workflow successfully processes the input data and returns:
- **Calculated BMI**: The exact BMI value
- **Category**: The appropriate classification based on WHO standards

### Key Benefits of This Approach:
1. **Type Safety**: TypedDict ensures correct data structure
2. **Modularity**: Separate functions for calculation and classification
3. **Scalability**: Easy to add more processing steps to the workflow
4. **Clarity**: Clear separation of concerns and workflow visualization

### Possible Extensions:
- Add input validation
- Include additional health metrics
- Add visualization of BMI categories
- Support for different measurement units (imperial/metric)

## 6. Example Execution

Let's test our BMI calculator with sample data:
- **Weight**: 105 kg
- **Height**: 1.70 m
- **Expected BMI**: ~36.3 (Obesity category)

The workflow will process the initial state through both calculation and classification steps.